In [28]:
import tensorflow as tf; print(f"\t\t– TENSORFLOW VERSION: {tf.__version__}");
import pandas as pd; pd.options.mode.chained_assignment = None;
import os
import ast
import cv2
import numpy as np
from glob import glob
from PIL import Image
import plotly.express as px
import matplotlib.pyplot as plt
from tqdm.notebook import tqdm; tqdm.pandas();

		– TENSORFLOW VERSION: 2.9.0


In [29]:
DATA_DIR = "D:Dataset/dataset/hubmap-organ-segmentation/"
for file in tf.io.gfile.glob(os.path.join(DATA_DIR, "*")): print(f"\t--> {file}")

	--> D:Dataset\dataset\hubmap-organ-segmentation\masked_images
	--> D:Dataset\dataset\hubmap-organ-segmentation\sample_submission.csv
	--> D:Dataset\dataset\hubmap-organ-segmentation\test.csv
	--> D:Dataset\dataset\hubmap-organ-segmentation\test_images
	--> D:Dataset\dataset\hubmap-organ-segmentation\train.csv
	--> D:Dataset\dataset\hubmap-organ-segmentation\train_annotations
	--> D:Dataset\dataset\hubmap-organ-segmentation\train_fix.csv
	--> D:Dataset\dataset\hubmap-organ-segmentation\train_images


In [30]:
train_df = pd.read_csv(DATA_DIR+"train.csv")
test_df = pd.read_csv(DATA_DIR+"test.csv")

In [31]:
train_df

,id,organ,data_source,img_height,img_width,pixel_size,tissue_thickness,rle,age,sex
0,10044,prostate,HPA,3000,3000,0.4,4,1459676 77 1462675 82 1465674 87 1468673 92 14...,37.0,Male
1,10274,prostate,HPA,3000,3000,0.4,4,715707 2 718705 8 721703 11 724701 18 727692 3...,76.0,Male
2,10392,spleen,HPA,3000,3000,0.4,4,1228631 20 1231629 24 1234624 40 1237623 47 12...,82.0,Male
3,10488,lung,HPA,3000,3000,0.4,4,3446519 15 3449517 17 3452514 20 3455510 24 34...,78.0,Male
4,10610,spleen,HPA,3000,3000,0.4,4,478925 68 481909 87 484893 105 487863 154 4908...,21.0,Female
...,...,...,...,...,...,...,...,...,...,...
346,9517,kidney,HPA,3000,3000,0.4,4,1611763 11 1614753 29 1617750 35 1620746 43 16...,61.0,Male
347,9769,kidney,HPA,3070,3070,0.4,4,4030400 28 4033466 34 4036526 48 4039594 54 40...,28.0,Male
348,9777,largeintestine,HPA,3000,3000,0.4,4,538473 13 541468 22 544463 30 547461 35 550459...,84.0,Male
349,9791,kidney,HPA,3000,3000,0.4,4,334733 33 337729 43 340729 43 343725 51 346723...,28.0,Male


In [32]:
train_df=train_df[["id", "organ","img_height","pixel_size"]]

In [35]:
train_img_path = [DATA_DIR+"train_images/"+str(x)+".tiff" for x in train_df.id.values]
mask_img_path = [DATA_DIR+"masked_images/"+str(x)+".jpg" for x in train_df.id.values]

In [36]:
train_df["mask_img_path"] = mask_img_path
train_df["train_img_path"] = train_img_path

In [41]:
train_df.to_csv(DATA_DIR+"train_fix.csv", index_label=False)

In [42]:
pd.read_csv(DATA_DIR+"train_fix.csv")

,id,organ,img_height,pixel_size,mask_img_path,train_img_path
0,10044,prostate,3000,0.4,D:Dataset/dataset/hubmap-organ-segmentation/ma...,D:Dataset/dataset/hubmap-organ-segmentation/tr...
1,10274,prostate,3000,0.4,D:Dataset/dataset/hubmap-organ-segmentation/ma...,D:Dataset/dataset/hubmap-organ-segmentation/tr...
2,10392,spleen,3000,0.4,D:Dataset/dataset/hubmap-organ-segmentation/ma...,D:Dataset/dataset/hubmap-organ-segmentation/tr...
3,10488,lung,3000,0.4,D:Dataset/dataset/hubmap-organ-segmentation/ma...,D:Dataset/dataset/hubmap-organ-segmentation/tr...
4,10610,spleen,3000,0.4,D:Dataset/dataset/hubmap-organ-segmentation/ma...,D:Dataset/dataset/hubmap-organ-segmentation/tr...
...,...,...,...,...,...,...
346,9517,kidney,3000,0.4,D:Dataset/dataset/hubmap-organ-segmentation/ma...,D:Dataset/dataset/hubmap-organ-segmentation/tr...
347,9769,kidney,3070,0.4,D:Dataset/dataset/hubmap-organ-segmentation/ma...,D:Dataset/dataset/hubmap-organ-segmentation/tr...
348,9777,largeintestine,3000,0.4,D:Dataset/dataset/hubmap-organ-segmentation/ma...,D:Dataset/dataset/hubmap-organ-segmentation/tr...
349,9791,kidney,3000,0.4,D:Dataset/dataset/hubmap-organ-segmentation/ma...,D:Dataset/dataset/hubmap-organ-segmentation/tr...
